# A neuron with calcium dynamics and spike-frequency adaptation due to a Ca2+-activated K+ channel

**Calcium is good for you!**
![image](https://media.istockphoto.com/photos/glass-of-milk-picture-id1206080627?k=20&m=1206080627&s=612x612&w=0&h=NfdmNI8WYa5Kd7zMCqpZ8hFkakQCWzkv9aD9r5yhdRw=)

## Step 1: Setup

In [ ]:
# Setup inline plotting
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
# For Google Colab, this line installs NEURON
!pip install neuron quantities

In [ ]:
# Fetch mechanisms
# Uncomment this line if on google colab
#!git clone https://github.com/ABL-Lab/NSC6084-A25.git

In [ ]:
# Compile the mechanisms
# Note: recompiled mechanisms will not take effect until neuron is imported or the jupyter kernel is restarted

# Uncomment this line if on google colab
#!nrnivmodl ./NSC6084-A25/Sept16/mechanisms
# Uncomment this line if running locally
!nrnivmodl mechanisms

In [ ]:
# We will let this library handle unit conversion for us
import quantities as pq
from quantities import um, nS, mV, cm, ms, nA, S, uF, Hz, degrees, s

In [ ]:
# Import and initialize NEURON
import neuron
from neuron import h
h.load_file("stdrun.hoc")

In [ ]:
# Import other modules we need
import numpy as np

## Step 2: Define the circuit
We will use a single compartment, called a "Section" (more on that in next lectures). <br>
It has a cylindrical geometry with length "L" and a diameter "diam", and a specific capacitance "cm" (capacitance per area) <br>
**Unit conversion is a common source of error, so we will be explicit with our units.** 

In [ ]:
soma = h.Section()

### Query NEURON for the expected units for soma.L & soma.diam

In [ ]:
[h.units(x) for x in ["L", "diam"]]

In [ ]:
soma.L = 20 * um
soma.diam =  20 * um

In [ ]:
volume = soma(0.5).volume() * um**3

In [ ]:
area = soma(0.5).area() * um**2

In [ ]:
area

In [ ]:
volume

### Assign the membrane capacitance "everywhere"

In [ ]:
h.units("cm")  # Query the expected units

In [ ]:
specific_membrane_capacitance = 1 * uF/cm**2

In [ ]:
for sec in soma.wholetree():
    sec.cm = specific_membrane_capacitance #  specific membrane capacitance (micro Farads / cm^2)
    sec.Ra = 100

### Add transient Na+, delayed rectified K+, leak

In [ ]:
# This model includes the transient Na+, persistent K+ and the leak conductances
soma.insert("pas")
soma.insert("NaTg")
soma.insert("K_Pst")

In [ ]:
h.celsius = 34

In [ ]:
soma(0.5).K_Pst.gK_Pstbar = 0.2
soma(0.5).NaTg.gNaTgbar = 0.42

### Parametize the leak conductance G = 1/R

In [ ]:
G = 2.0 * nS  # R = 1/G in our RC circuit

In [ ]:
v_rest = -70*mV

In [ ]:
tau_m = (specific_membrane_capacitance * area / G).rescale(ms)

In [ ]:
# Assign the leak conductance everywhere
for seg in soma:
    seg.pas.g = (G/area).rescale(S/cm**2)  # Compute specific conductance, and rescale to units of 'S/cm2'
    seg.pas.e = -70.0

In [ ]:
tau_m = ((soma(0.5).cm * uF/cm**2) / (soma(0.5).pas.g * S/cm**2)).rescale(ms)

In [ ]:
tau_m

### Inspect our parameters

In [ ]:
soma.psection()

In [ ]:
soma.nseg

### Add a current injection

In [ ]:
stim = h.IClamp(soma(0.5))

In [ ]:
stim.delay = 200 * ms  # Inject current 500ms after the start of the simulation 
stim.dur = 600 * ms  # stop injecting current at 520ms 
stim.amp = 0.025 * nA  # Inject 0.1 nA of current

## Step 3: Run the simulation

### Define recordings of simulation variables

In [ ]:
soma_v = h.Vector().record(soma(0.5)._ref_v)
t = h.Vector().record(h._ref_t)

### Run the simulation

In [ ]:
h.finitialize( float(v_rest) )
h.continuerun( float(1000 * ms) )

## Step 4: Plot the results

In [ ]:
plt.plot(t, soma_v, lw=2, label="soma(0.5).v")
plt.legend(fontsize=12)
plt.xlabel("t [ms]", size=16)
plt.ylabel("v [mV]", size=16)
plt.xticks(size=12)
plt.yticks(size=12)
#plt.axis([0,1000,-80,50])

In [ ]:
def find_spikes(v, t):
    """ Returns times of spikes for a voltage trace and time grid"""
    # look for upward crossing of 0mV
    v_arr = np.array(v)
    t_arr = np.array(t) 
    # This is tricky & powerful notation! Let's discuss in class!
    return t_arr[1:][(v_arr[1:]>0) & (v_arr[:-1]<0)] 

In [ ]:
I_range = np.arange(0,0.15,0.005)

In [ ]:
def find_freq(I):
    stim.amp = I
    h.finitialize( float(v_rest) )
    h.continuerun( float(1000 * ms) )
    spike_times = find_spikes(soma_v, t)
    firing_freq = (len(spike_times)/(stim.dur*ms)).rescale(Hz)
    return firing_freq

In [ ]:
# Note this cool notation: List comprehension
freqs = [find_freq(x) for x in I_range]

In [ ]:
plt.plot(I_range, freqs, 'x')
#plt.axis([0.01, 0.02, 0, 100])

In [ ]:
#soma.uninsert("K_Tst")
#soma(0.5).K_Tst.gK_Tstbar = 0.0

## Adding calcium dynamics

### 1) Add extrusion and buffering

In [ ]:
soma.insert("CaDynamics_DC0")

In [ ]:
soma(0.5).CaDynamics_DC0.decay = 100

### Add the calcium channels

In [ ]:
soma.insert("Ca_HVA2")

In [ ]:
soma(0.5).Ca_HVA2.gCa_HVAbar = 0.005

In [ ]:
#soma.insert("Ca_LVAst")

In [ ]:
#soma(0.5).Ca_LVAst.gCa_LVAstbar = 0.0

### Insert an SK-type Ca2+ activated potassium channel

In [ ]:
soma.insert("SK_E2")

In [ ]:
soma(0.5).SK_E2.gSK_E2bar = 0.02

### Record the calcium and SK conductance

In [ ]:
cai = h.Vector().record(soma(0.5)._ref_cai)
gske2 = h.Vector().record(soma(0.5).SK_E2._ref_gSK_E2 )

In [ ]:
# Watch out for units
h.units("cai")

In [ ]:
stim.amp = 1.0 * nA  # Inject 0.1 nA of current

In [ ]:
h.finitialize( float(v_rest) )
h.continuerun( float(1000 * ms) )

In [ ]:
plt.plot(t, cai*1000, lw=2, label="soma(0.5).cai")
plt.legend(fontsize=12)
plt.xlabel("t [ms]", size=16)
plt.ylabel("cai [uM]", size=16)
plt.xticks(size=12)
plt.yticks(size=12)
#plt.axis([100,200,-80,30])

In [ ]:
plt.plot(t, gske2, lw=2, label="soma(0.5).gSK_E2")
plt.legend(fontsize=12)
plt.xlabel("t [ms]", size=16)
plt.ylabel("specific conductance [S/cm2]", size=16)
plt.xticks(size=12)
plt.yticks(size=12)
#plt.axis([100,200,-80,30])

In [ ]:
plt.plot(t, soma_v, lw=2, label="soma(0.5).v")
plt.legend(fontsize=12)
plt.xlabel("t [ms]", size=16)
plt.ylabel("v [mV]", size=16)
plt.xticks(size=12)
plt.yticks(size=12)
plt.axis([100,500,-80,50])

## Now it's your turn!

### **Question 1** 
Create a function to return the firing frequency as 1/inter-spike interval for a given current input I, and plot the firing frequency for a range of currents 

Now compare that firing frequencies computed from the 1st, 2nd or last inter-spike interval

What is the role of the SK channel? What happens if it is blocked (g = 0)?

